In [1]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

ds = load_dataset("toughdata/quora-question-answer-dataset", split="train")

Generating train split:   0%|          | 0/56402 [00:00<?, ? examples/s]

In [4]:
ds = ds.train_test_split(test_size=0.2)

In [5]:
ds["train"][0]

{'question': 'Have you ever been judged for your appearance?',
 'answer': "Yea, that's the reason I have heavy body image issues to this day. I find myself comparing my looks to everyone I see because I feel like I'm way too ugly. These are unhealthy thoughts and they led me to an extremely dark place. It hurts to be called ugly etc. I hate when people feel the need to bodyshame or judge others. It's disgusting. It hurts even more when its from somebody you love and im speaking from personal experience. On a scale of 1 to 10 I'm a 2 🌈 😀🤚 in my opinion. Excuse this poorly written answer, I'm not really in the mood to write. Fuck everyone who says your ugly is the life message here because your not, you're special, precious, and beautiful.\n"}

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# def formatting_prompts_func(example):
#     output_texts = {}
#     for i in range(len(example['question'])):
#         text = f"### Question: {example['question'][i]}\n ### Answer: {example['answer'][i]}"
#         output_texts.update(text)
#     return output_texts

# processed_ds = ds.map(formatting_prompts_func, batched=True)

prefix = "Question: "

 
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["question"]]
    inputs = [doc + " Answer: " + doc2 for doc,doc2 in zip(inputs,examples["answer"])]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    return model_inputs

tokenized_ds = ds.map(preprocess_function, batched=True,remove_columns=ds["train"].column_names, )

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/45121 [00:00<?, ? examples/s]

Map:   0%|          | 0/11281 [00:00<?, ? examples/s]

In [7]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [8]:
lm_dataset = tokenized_ds.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/45121 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/11281 [00:00<?, ? examples/s]

In [9]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

2024-07-30 07:26:17.234657: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 07:26:17.234767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 07:26:17.395561: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("bert-base-uncased", is_decoder=True)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [11]:
training_args = TrainingArguments(
    output_dir="bert-Causal-QA",
    eval_strategy="epoch",
    learning_rate=2e-5,
    save_total_limit=3,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
#     predict_with_generate=True,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
#     compute_metrics = compute_metrics
)

In [12]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,3.817100,3.581651
2,3.508700,3.399627
3,3.394600,3.357927


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=6312, training_loss=3.721778608546964, metrics={'train_runtime': 2534.2122, 'train_samples_per_second': 39.84, 'train_steps_per_second': 2.491, 'total_flos': 6643421077478400.0, 'train_loss': 3.721778608546964, 'epoch': 3.0})

In [13]:
import math

eval_results = trainer.evaluate()
perplexity = math.exp(eval_results['eval_loss'])
eval_results['perplexity'] = perplexity

In [14]:
print(eval_results)

{'eval_loss': 3.357926845550537, 'eval_runtime': 64.3624, 'eval_samples_per_second': 130.868, 'eval_steps_per_second': 8.188, 'epoch': 3.0, 'perplexity': 28.729568264873752}


In [15]:
! git clone https://huggingface.co/spaces/karanzrk/bert-causal-quora-qa

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Cloning into 'bert-causal-quora-qa'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 33 (delta 13), reused 1 (delta 0), pack-reused 5 (from 1)
Unpacking objects: 100% (33/33), 6.60 KiB | 750.00 KiB/s, done.
